### MGR Models

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
import numpy as np
import pandas as pd
import os
import pathlib
import matplotlib.pyplot as plt
import random
from PIL import Image
import IPython.display as ipd
from tqdm.autonotebook import tqdm

import librosa
import librosa.display

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
import torchvision.models as models
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

%matplotlib inline


In [32]:
train_root = '/content/drive/MyDrive/colab_music/train/'
val_root = '/content/drive/MyDrive/colab_music/test/'

train_data = torchvision.datasets.ImageFolder(root=train_root, 
                                              transform=T.Compose([
                                                        T.Resize(265),
                                                        T.RandomRotation(20),
                                                        T.RandomHorizontalFlip(),
                                                        T.ToTensor()]))

val_data = torchvision.datasets.ImageFolder(root=val_root, 
                                            transform=T.Compose([  
                                                      T.Resize(265),  
                                                      T.ToTensor()]))


In [33]:
torch.manual_seed(21)
val_size = int(0.2 * len(train_data))
train_size = len(train_data) - val_size

train_ds, val_ds = random_split(train_data, [train_size,val_size])

train_dl = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=5)
val_dl = DataLoader(val_ds, batch_size=16, num_workers=5)

print(f'Training size: {len(train_ds)}, Validation size: {len(val_ds)}')

Training size: 5603, Validation size: 1400


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [34]:
class Custom_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3),
            nn.BatchNorm2d(num_features=8),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(8, 16, kernel_size=3, stride=1),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(16, 32, kernel_size=3, stride=1),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.AdaptiveAvgPool2d(1),

            nn.Flatten(),
            nn.Dropout(p=0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 10))
        
    def forward(self, xb):
        return self.network(xb)
    
class Resenet_34(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = models.resnet34(pretrained=True)
        num_ftrs = self.network.fc.in_features
        self.network.fc = nn.Linear(num_ftrs, 10)
    
    def forward(self, xb):
        return self.network(xb)

    def freeze(self):
      for param in self.network.parameters():
        param.require_grad = False
      for param in self.network.fc.parameters():
        param.require_grad = True
    
    def unfreeze(self):
      for param in self.network.parameters():
        param.require_grad = True

In [35]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
        
def one_pass(epochs, learing_rate, model, train_loader, val_loader, 
                  weight_decay, grad_clip, optim):
    torch.cuda.empty_cache()
    iterations = []
    optimizer = optim(model.parameters(), learing_rate, weight_decay=weight_decay)
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, learing_rate, epochs=epochs, steps_per_epoch=len(train_loader))
    
    for epoch in range(epochs):
        
        model.train()
        learning_rates = []

        for batch in tqdm(train_loader):

            x, y = batch
            predict_train = model(x)
            loss = F.cross_entropy(predict_train, y)

            loss.backward()

            if grad_clip: 
              nn.utils.clip_grad_value_(model.parameters(), grad_clip)

            optimizer.step()
            optimizer.zero_grad()

            for parameter_g in optimizer.param_groups:
              learning_rates.append(parameter_g['lr'])
            
            sched.step()

        outputs = []
        for batch in val_loader:
          x, y = batch
          predict_val = model(x)
          loss = F.cross_entropy(predict_val, y)
          prediction_val = torch.max(predict_val, dim=1)[1]
          accuracy_torch = torch.tensor(torch.sum(prediction_val == y).item() / len(prediction_val))
          outputs.append({'validation loss': loss.detach(), 'validation accuracy': accuracy_torch.detach() })


        loss_b = [x['validation loss'] for x in outputs]
        loss_mean = torch.stack(loss_b).mean()
        score_b = [x['validation accuracy'] for x in outputs]
        score_mean = torch.stack(score_b).mean()
        result = {'validation loss': loss_mean.item(), 'validation accuracy': score_mean.item()}

        result['lrs'] = learning_rates

        print(f"Epoch [{epoch}], \
        validation loss: {result['validation loss']:.2f}, \
        validation accuracy: {result['validation accuracy']:.2f}")

        iterations.append(result)
    return iterations

In [36]:
def to_cuda(dataset, cuda_d):
    if isinstance(dataset, (list,tuple)):
        return [to_cuda(x, cuda_d) for x in dataset]
    return dataset.to(cuda_d, non_blocking=True)

class CudaDataLoader():
    def __init__(self, dataloader, cuda_set):
        self.dataloader = dataloader
        self.cuda_set = cuda_set
        
    def __iter__(self):
        for b in self.dataloader: 
            yield to_cuda(b, self.cuda_set)

    def __len__(self):
        return len(self.dataloader)

In [37]:
train_dl = CudaDataLoader(train_dl, torch.device('cuda'))
val_dl = CudaDataLoader(val_dl, torch.device('cuda'))

In [38]:
# Custom CNN model

model = to_cuda(Custom_CNN(), torch.device('cuda'))

torch.cuda.empty_cache()
    
outputs = []
for batch in val_dl:
  x, y = batch
  predict_val = model(x)
  loss = F.cross_entropy(predict_val, y)
  prediction_val = torch.max(predict_val, dim=1)[1]
  accuracy_torch = torch.tensor(torch.sum(prediction_val == y).item() / len(prediction_val))
  outputs.append({'validation loss': loss.detach(), 'validation accuracy': accuracy_torch.detach() })


loss_b = [x['validation loss'] for x in outputs]
loss_mean = torch.stack(loss_b).mean()
score_b = [x['validation accuracy'] for x in outputs]
score_mean = torch.stack(score_b).mean()
iterations = [{'validation loss': loss_mean.item(), 'validation accuracy': score_mean.item()}]

iterations += one_pass(epochs=13, learing_rate=0.001, model=model, 
                         train_loader=train_dl, val_loader=val_dl,
                         grad_clip=0.1,
                         weight_decay=1e-4,
                         optim=torch.optim.Adam)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Epoch [0],         validation loss: 1.75,         validation accuracy: 0.33



Epoch [1],         validation loss: 1.39,         validation accuracy: 0.46



Epoch [2],         validation loss: 1.22,         validation accuracy: 0.56



Epoch [3],         validation loss: 1.15,         validation accuracy: 0.59


...


Epoch [8],         validation loss: 0.81,         validation accuracy: 0.71



Epoch [9],         validation loss: 0.78,         validation accuracy: 0.72



Epoch [10],         validation loss: 0.75,         validation accuracy: 0.74



Epoch [11],         validation loss: 0.73,         validation accuracy: 0.75



Epoch [12],         validation loss: 0.74,         validation accuracy: 0.74


In [39]:
# Resnet model

%%time

model = to_cuda(Resenet_34(), torch.device('cuda'))

torch.cuda.empty_cache()

outputs = []
for batch in val_dl:
  x, y = batch
  predict_val = model(x)
  loss = F.cross_entropy(predict_val, y)
  prediction_val = torch.max(predict_val, dim=1)[1] 
  accuracy_torch = torch.tensor(torch.sum(prediction_val == y).item() / len(prediction_val))
  outputs.append({'validation loss': loss.detach(), 'validation accuracy': accuracy_torch.detach() })


loss_b = [x['validation loss'] for x in outputs]
loss_mean = torch.stack(loss_b).mean()
score_b = [x['validation accuracy'] for x in outputs]
score_mean = torch.stack(score_b).mean()
iterations = [{'validation loss': loss_mean.item(), 'validation accuracy': score_mean.item()}]

iterations += one_pass(epochs=13, learing_rate=0.001, model=model, 
                         train_loader=train_dl, val_loader=val_dl,
                         grad_clip=0.1,
                         weight_decay=1e-4,
                         optim=torch.optim.Adam)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Epoch [0],         validation loss: 0.97,         validation accuracy: 0.65



Epoch [1],         validation loss: 1.03,         validation accuracy: 0.64



Epoch [2],         validation loss: 0.93,         validation accuracy: 0.65



Epoch [3],         validation loss: 0.72,         validation accuracy: 0.75


...


Epoch [8],         validation loss: 0.39,         validation accuracy: 0.87



Epoch [9],         validation loss: 0.32,         validation accuracy: 0.89



Epoch [10],         validation loss: 0.25,         validation accuracy: 0.91



Epoch [11],         validation loss: 0.21,         validation accuracy: 0.93



Epoch [12],         validation loss: 0.20,         validation accuracy: 0.94
CPU times: user 11min 29s, sys: 6min 20s, total: 17min 50s
Wall time: 21min 59s
